<h4>Intro to TF</h4>
<p>Document how the input stages using tf.nn.conv2d have to match the dimensions using filters
and strides. Google Vincent does a good job. Document this here. </p>

In [ ]:
#MNIST TF not CNN

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


In [ ]:
#dnn mnist

import time
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print ('%s function took %0.3f ms' % (f.func_name, (time2-time1)*1000.0))
        return ret
    return wrap



time1 = time.time()
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#sess = tf.Session() add session=sess
sess.run(tf.global_variables_initializer())
for i in range(4000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(session=sess,feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(session=sess,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(session=sess,feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

time2 = time.time()
print ('minutes:' , (time2-time1)/60)

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
import numpy as np
from keras.utils import np_utils
from keras.layers import Activation,Dense,Dropout
from keras.models import Sequential

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)
 
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
 
print(X_train.shape, y_train.shape, X_test.shape,y_test.shape,Y_train.shape,Y_test.shape)

model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_train, Y_train,
          batch_size=128, nb_epoch=10,
          show_accuracy=True, verbose=1,
          validation_data=(X_test, Y_test))

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")



In [ ]:
from keras.models import model_from_json


json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [22]:
#CNN KERAS
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
import numpy as np
from keras.utils import np_utils
from keras.layers import Activation,Dense,Dropout,Flatten
from keras.optimizers import SGD
from keras.models import Sequential

num_class = 10  # number of class

input_shape = (1, 28, 28)
def preprocess_input(x):
    return x.reshape((-1, ) + input_shape) / 255.


def preprocess_output(y):
    return np_utils.to_categorical(y)

(train_x, train_y), (validation_x, validation_y) = mnist.load_data()
train_x, validation_x = map(preprocess_input, [train_x, validation_x])
train_y, validation_y = map(preprocess_output, [train_y, validation_y])
print('Loading MNIST data...')
print('train_x shape:', train_x.shape, 'train_y shape:', train_y.shape)
print('validation_x shape:', validation_x.shape,
      'validation_y shape', validation_y.shape)

print("Training matrix shape", train_x.shape,train_y.shape, validation_x.shape, validation_y.shape)

Y_train = np_utils.to_categorical(train_y, 10)
Y_test = np_utils.to_categorical(validation_y, 10)
 
print(train_x.shape, train_y.shape,validation_x.shape,validation_y.shape,Y_train.shape, Y_test.shape)

model = Sequential()

model.add(Conv2D(32,3, input_shape= (1,28,28),padding='same', name='conv1'))
model.add(MaxPooling2D(2, name='pool1', data_format="channels_first"))
model.add(Conv2D(64, 3, padding='same', name='conv2'))
model.add(MaxPooling2D(2, name='pool2', data_format="channels_first"))
model.add(Flatten(name='flatten'))
model.add(Dense(64, activation='relu', name='fc1'))
model.add(Dense(num_class, activation='softmax', name='fc2'))
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(lr=0.01, momentum=0.9),
            metrics=['accuracy'])

model.fit(train_x, train_y,
          batch_size=128, nb_epoch=10,verbose=1,
          validation_data=(validation_x, validation_y))

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")



Loading MNIST data...
train_x shape: (60000, 1, 28, 28) train_y shape: (60000, 10)
validation_x shape: (10000, 1, 28, 28) validation_y shape (10000, 10)
Training matrix shape (60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)
(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10) (600000, 10) (100000, 10)


/Users/dc/anaconda/envs/tf35/lib/python3.5/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s - loss: 0.5819 - acc: 0.8262 - val_loss: 0.1897 - val_acc: 0.9440
Epoch 2/10
60000/60000 [==============================] - 10s - loss: 0.1581 - acc: 0.9539 - val_loss: 0.1155 - val_acc: 0.9645
Epoch 3/10
60000/60000 [==============================] - 10s - loss: 0.1148 - acc: 0.9653 - val_loss: 0.1039 - val_acc: 0.9674
Epoch 4/10
60000/60000 [==============================] - 10s - loss: 0.0919 - acc: 0.9722 - val_loss: 0.0789 - val_acc: 0.9746
Epoch 5/10
60000/60000 [==============================] - 11s - loss: 0.0779 - acc: 0.9759 - val_loss: 0.0741 - val_acc: 0.9765
Epoch 6/10
60000/60000 [==============================] - 11s - loss: 0.0684 - acc: 0.9788 - val_loss: 0.0667 - val_acc: 0.9775
Epoch 7/10
60000/60000 [==============================] - 11s - loss: 0.0605 - acc: 0.9810 - val_loss: 0.0567 - val_acc: 0.9818
Epoch 8/10
60000/60000 [==============================

In [8]:
import keras
print(keras.__version__)

2.0.6
